In [1]:
import pandas as pd
import mysql.connector
import json
import pymongo
from funciones import *
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from empiricaldist import Cdf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model, svm
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, TweedieRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from yellowbrick.regressor import ResidualsPlot
import pygad
import random
import math
import matplotlib.pyplot as plt

In [2]:
with open('Modelos/Comp_a/RandomForestRegressor_modelo.pkl', 'rb') as archivo:
    modelo_a = pickle.load(archivo)
with open('Modelos/Comp_b/RandomForestRegressor_modelo.pkl', 'rb') as archivo:
    modelo_b = pickle.load(archivo)
with open('Modelos/Comp_c/RandomForestRegressor_modelo.pkl', 'rb') as archivo:
    modelo_c = pickle.load(archivo)
with open('Modelos/Comp_d/RandomForestRegressor_modelo.pkl', 'rb') as archivo:
    modelo_d = pickle.load(archivo)
with open('Modelos/Comp_a/scaler_model.pkl', 'rb') as archivo:
    scaler_a = pickle.load(archivo)
with open('Modelos/Comp_b/scaler_model.pkl', 'rb') as archivo:
    scaler_b = pickle.load(archivo)
with open('Modelos/Comp_c/scaler_model.pkl', 'rb') as archivo:
    scaler_c = pickle.load(archivo)
with open('Modelos/Comp_d/scaler_model.pkl', 'rb') as archivo:
    scaler_d = pickle.load(archivo)
comp_a = pd.read_csv('Datos/Originales/Comp_A.csv')
comp_b = pd.read_csv('Datos/Originales/Comp_B.csv')
comp_c = pd.read_csv('Datos/Originales/Comp_C.csv')
comp_d = pd.read_csv('Datos/Originales/Comp_D.csv')
condiciones_iniciales = [25, 25, 25, 25]
# Generamos la poblacion inicial
random.seed(42) #semilla para que en todas las ejecuciones la poblacion inicial sea la misma
poblacion_inicial = []
poblacion_inicial.append(condiciones_iniciales)
for _ in range(50):
    individuo = []
    for _ in range(4):
        gen = random.uniform(1, 90)
        individuo.append(gen)
    poblacion_inicial.append(individuo)

c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.4.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.0 wh

In [3]:
presion = list(comp_a['Presion'])
presion.extend(comp_b['Presion'])
presion.extend(comp_c['Presion'])
presion.extend(comp_d['Presion'])
presion = np.median(presion)
temp = list(comp_a['Temperatura'])
temp.extend(comp_b['Temperatura'])
temp.extend(comp_c['Temperatura'])
temp.extend(comp_d['Temperatura'])
temp = np.median(temp)

In [4]:
#definimos la funcion fitness
def fitness_func(ga_instance, solution, solution_idx):
    capacidad = np.array([100, 90, 95, 110])
    frecuencia_porcentaje = [numero / 100 for numero in solution]
    volumen_aire  = capacidad * frecuencia_porcentaje
    if sum(volumen_aire) >= objetivo:
        comp_a = pd.DataFrame({
        'Presion': [presion],
        'Temperatura': [temp],
        'Frecuencia': solution[0]
        })
        X = scaler_d.transform(comp_a)
        X = pd.DataFrame(X, columns=comp_a.columns)
        potencia_a = modelo_a.predict(X)
        comp_b = pd.DataFrame({
        'Presion': [presion],
        'Temperatura': [temp],
        'Frecuencia': solution[1]
        })
        X = scaler_d.transform(comp_b)
        X = pd.DataFrame(X, columns=comp_b.columns)
        potencia_b = modelo_b.predict(X)
        comp_c = pd.DataFrame({
        'Presion': [presion],
        'Temperatura': [temp],
        'Frecuencia': solution[2]
        })
        X = scaler_d.transform(comp_c)
        X = pd.DataFrame(X, columns=comp_c.columns)
        potencia_c = modelo_c.predict(X)
        comp_d = pd.DataFrame({
        'Presion': [presion],
        'Temperatura': [temp],
        'Frecuencia': solution[3]
        })
        X = scaler_d.transform(comp_d)
        X = pd.DataFrame(X, columns=comp_d.columns)
        potencia_d = modelo_d.predict(X)
        fitness = (potencia_a + potencia_b + potencia_c + potencia_d)
        penalizacion = sum(volumen_aire) - objetivo
        fitness = (fitness)+math.exp(penalizacion)
        for i in range(len(solution)):
            if solution[i] != condiciones_iniciales[i]:
                fitness +=3
        fitness = -fitness
        fitness = float(fitness[0])
    else:
        fitness = -10e99
    return fitness

In [5]:
num_generations = 100000
num_parents_mating = len(poblacion_inicial)//2
num_genes = len(poblacion_inicial[0])
parent_selection_type = 'random'
crossover_type = 'two_points'
mutation_type = 'swap'
mutation_percent_genes = 1
mutation_probability = 0.1
keep_parents = 2
gene_space = range(1, 90)

In [6]:
def on_stop(ga_instance, last_population_fitness):
    print(f"El algoritmo genético ha finalizado.")
    print("Mejor solución encontrada:", ga_instance.best_solution())

In [7]:
def on_generation(ga_instance):
    print("Generación {generation}: Mejor solucion = {best_solution} Mejor fitness = {best_fitness}".format(generation=ga_instance.generations_completed,
                                                                          best_solution = ga_instance.best_solution()[0],
                                                                          best_fitness = ga_instance.best_solution()[1]))

In [8]:
def nuevos_datos(consigna):
    global objetivo
    objetivo = consigna
    ga_instance_1 = pygad.GA(fitness_func=fitness_func,
                       num_generations=100000, 
                       num_parents_mating=num_parents_mating,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       crossover_type=crossover_type,
                       mutation_type= mutation_type,
                       gene_space=gene_space,
                       mutation_percent_genes = mutation_percent_genes,
                       mutation_probability=mutation_probability,
                       initial_population=poblacion_inicial,
                       gene_type=float,
                       on_stop=on_stop,
                       save_solutions=True,
                       stop_criteria="saturate_20")
    ga_instance_1.run()
    #eliminamos la penalizacion del fitness para obtener la potencia medida
    capacidad = np.array([100, 90, 95, 110])
    frecuencia_porcentaje = [numero / 100 for numero in list(ga_instance_1.best_solution()[0])]
    volumen_aire  = capacidad * frecuencia_porcentaje
    penalizacion = sum(volumen_aire) - objetivo
    potencia = (ga_instance_1.best_solution()[1])+math.exp(penalizacion)
    return objetivo, ga_instance_1.best_solution()[0][0], ga_instance_1.best_solution()[0][1], ga_instance_1.best_solution()[0][2], ga_instance_1.best_solution()[0][3], -potencia

In [9]:
random.seed(42)  # Semilla para que siempre se generen las mismas consignas
lista = []
consignas_generadas = set()
while len(lista) < 20:
    consigna = random.randint(150, 250)
    if consigna not in consignas_generadas:
        consignas_generadas.add(consigna)
        datos = nuevos_datos(consigna)
        lista.append(datos)

c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1162: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in

El algoritmo genético ha finalizado.
Mejor solución encontrada: (array([86.19196343, 25.79051788, 72.83441632, 48.20147683]), -471.5405298582343, 0)


c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1162: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in

El algoritmo genético ha finalizado.
Mejor solución encontrada: (array([84.26076741, 16.90001737, 48.72430014, 16.90001737]), -423.0724162284589, 0)


c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1162: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in

...

El algoritmo genético ha finalizado.
Mejor solución encontrada: (array([70.2977566 , 34.9641165 , 19.77747308, 85.27850063]), -454.3510636327318, 0)


c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1162: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in

El algoritmo genético ha finalizado.
Mejor solución encontrada: (array([78.99671876, 16.90001737, 82.2238777 , 49.93251982]), -468.03925167685543, 0)


c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")
c:\Users\HP\.conda\envs\reto06_progra\lib\site-packages\pygad\pygad.py:1162: UserWarning: Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in population is large.
  warnings.warn("Use the 'save_solutions' parameter with caution as it may cause memory overflow when either the number of generations, number of genes, or number of solutions in

El algoritmo genético ha finalizado.
Mejor solución encontrada: (array([48.72430014, 83.68977685,  2.02181095, 87.60730299]), -461.1331269674808, 0)


Se ha decidido utilizar mysql por delante de mongo ya que en los datos que se iban a utilizar todas las filas iban a contar con las mismas columnas por lo que la flexibilidad que nos ofrece mongo no es neccesaria. Por otro lado podiamos utilizar una clave primaria para que no se pudieran repetir consignas, haciendo mas sencillas las busquedas de consignas en las mismas. Además, es una base de datos mas utilizada por lo que la integración con las tecnologias del cliente seguramente sea mas sencilla.

In [16]:
keyfile = '.pwd.json' #hay que utilizar la utentificacion de cada uno
with open(keyfile, 'r') as f:
  creds = json.load(f)

usuario = creds.get("usuario")
contrasena = creds.get("contrasena")
ruta = creds.get("ruta")
conn = mysql.connector.connect(user=usuario, password=contrasena, host=ruta)
cur = conn.cursor()
cur.execute('''DROP DATABASE IF EXISTS compresores''') 
cur.execute('''CREATE DATABASE compresores''') 
conn.close() 
DATABASE = 'compresores'
conn = mysql.connector.connect(user=usuario, password=contrasena, host=ruta, database = DATABASE)
cur = conn.cursor()
try:
    cur.execute('''CREATE TABLE optimizados (
    Consigna INT NOT NULL,
    Compresor_A FLOAT NOT NULL,
    Compresor_B FLOAT NOT NULL,
    Compresor_C FLOAT NOT NULL,
    Compresor_D FLOAT NOT NULL,
    Potencia FLOAT NOT NULL,
    PRIMARY KEY (Consigna)           
    )''') 
except mysql.connector.Error as err:
    print(err)
    conn.rollback() 
finally:
    conn.close()
conn = mysql.connector.connect(user=usuario, password=contrasena, host=ruta, database = DATABASE)
cur = conn.cursor()

try:
    cur.executemany('''INSERT INTO optimizados (Consigna, Compresor_A, Compresor_B, Compresor_C, Compresor_D, Potencia) 
    VALUES (%s, %s, %s, %s, %s, %s)''',
    lista) 
    conn.commit()
except mysql.connector.Error as err:
    print(err)
    conn.rollback() 
finally:
    conn.close()

Querys

In [17]:
#miramos la base de datos entera
conn = mysql.connector.connect(user=usuario, password=contrasena, host=ruta, database = DATABASE)
cur = conn.cursor()
cur.execute('''SELECT * FROM optimizados''')
print(cur.fetchall())
conn.close()

[(153, 19.7775, 3.65195, 38.551, 84.919, 410.696), (154, 53.0282, 25.0, 8.10149, 65.1442, 414.062), (161, 87.426, 49.4998, 19.7775, 9.75919, 420.732), (163, 19.7775, 24.8289, 29.0753, 84.919, 416.647), (164, 84.2608, 16.9, 48.7243, 16.9, 421.655), (167, 43.3001, 25.0, 77.6889, 25.0, 423.383), (177, 89.9361, 3.65195, 78.9967, 8.73756, 416.797), (178, 89.9361, 5.19336, 9.09094, 68.6936, 413.003), (179, 86.192, 6.15534, 86.192, 6.15534, 417.768), (181, 82.2239, 25.0, 25.0, 48.2015, 423.761), (185, 63.7069, 30.9569, 65.8877, 28.7428, 451.606), (204, 73.0393, 29.0063, 86.192, 20.9981, 447.715), (214, 70.2978, 34.9641, 19.7775, 85.2785, 452.917), (219, 9.62684, 85.8896, 41.3814, 85.8896, 467.209), (221, 48.7243, 83.6898, 2.02181, 87.6073, 457.337), (225, 80.404, 8.73756, 89.6548, 48.0912, 457.666), (227, 78.9967, 16.9, 82.2239, 49.9325, 466.761), (231, 86.192, 25.7905, 72.8344, 48.2015, 469.686), (236, 89.5683, 65.1442, 24.7474, 58.8391, 476.819), (244, 78.9967, 25.0, 74.817, 65.1442, 477.27